In [1]:
from tensorflow import keras
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [51]:
housing=fetch_california_housing()

X_train_full,X_test,y_train_full,y_test=train_test_split(housing.data,housing.target)
X_train,X_valid,y_train,y_valid=train_test_split(X_train_full,y_train_full)

In [52]:
scaler=StandardScaler() 
X_train=scaler.fit_transform(X_train)
X_valid=scaler.transform(X_valid)
X_test=scaler.transform(X_test)
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(11610, 8)
(3870, 8)
(5160, 8)


### StandardScalerは標準化で，MinMaxScalerは正規化のモジュール
### fit : パラメータ（平均や標準偏差など）を計算
### transform : パラメータをもとにデータ変換
### fit_transform : パラメータ計算とデータ変換をまとめて実行
- 今回は，最初にfit_transformをすることでパラメータ計算とデータ変換を実行．計算したパラメータをscalerが保持してるので，その後に続く処理ではデータ変換だけでよい

In [53]:
input_ =keras.layers.Input(shape=X_train.shape[1:])#X_train.shape[1:]=8
hidden1=keras.layers.Dense(30,activation="relu")(input_)
hidden2=keras.layers.Dense(30,activation="relu")(hidden1)
concat=keras.layers.Concatenate()([input_,hidden2])
output=keras.layers.Dense(1)(concat)
model=keras.Model(inputs=[input_],outputs=[output])
print(input_.shape)
print(hidden1.shape)
print(hidden2.shape)
print(concat.shape)

(None, 8)
(None, 30)
(None, 30)
(None, 38)


### 入力が２つで，出力が1つの場合

In [54]:
input_A=keras.layers.Input(shape=[5],name='wide_input')
input_B=keras.layers.Input(shape=[6],name="deep_input")
hidden1=keras.layers.Dense(30,activation="relu")(input_B)
hidden2=keras.layers.Dense(30,activation="relu")(hidden1)
concat=keras.layers.concatenate([input_A,hidden2])
output=keras.layers.Dense(1,name="output")(concat)
model=keras.Model(inputs=[input_A,input_B],outputs=[output])
model.compile(loss="mse",optimizer=keras.optimizers.SGD(lr=0.001))

In [55]:
X_train_A,X_train_B=X_train[:,:5],X_train[:,2:]
X_valid_A,X_valid_B=X_valid[:,:5],X_valid[:,2:]
X_test_A,X_test_B  =X_test[:,:5],X_test[:,2:]
X_new_A,X_new_B=X_test_A[:3],X_test_B[:3]
print(X_train_A.shape)

(11610, 5)


In [57]:
history=model.fit((X_train_A,X_train_B),y_train,epochs=20,validation_data=((X_valid_A,X_valid_B),y_valid))
mse_test=model.evaluate((X_test_A,X_test_B),y_test)
y_pred=model.predict((X_new_A,X_new_B))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 0s 38us/sample - loss: 0.4769 - val_loss: 0.4930
Epoch 2/20
11610/11610 [==============================] - 0s 37us/sample - loss: 0.4742 - val_loss: 0.4927
Epoch 3/20
11610/11610 [==============================] - 0s 38us/sample - loss: 0.4727 - val_loss: 0.4879
Epoch 4/20
11610/11610 [==============================] - 0s 36us/sample - loss: 0.4707 - val_loss: 0.4881
Epoch 5/20
11610/11610 [==============================] - 0s 36us/sample - loss: 0.4689 - val_loss: 0.4850
Epoch 6/20
11610/11610 [==============================] - 0s 36us/sample - loss: 0.4678 - val_loss: 0.4830
Epoch 7/20
11610/11610 [==============================] - 0s 38us/sample - loss: 0.4660 - val_loss: 0.4828
Epoch 8/20
11610/11610 [==============================] - 0s 36us/sample - loss: 0.4648 - val_loss: 0.4798
Epoch 9/20
11610/11610 [==============================] - 0s 36us/sample - loss: 0.4630 - val_l

### 入力２つ，出力も2つの場合

In [40]:
output=keras.layers.Dense(1,name="main_output")(concat)
aux_output=keras.layers.Dense(1,name="aux_output")(hidden2)
model=keras.Model(inputs=[input_A,input_B],outputs=[output,aux_output])

In [41]:
model.compile(loss=["mse","mse"],loss_weights=[0.9,0.1],optimizer="sgd")
history=model.fit([X_train_A,X_train_B],[y_train,y_train],epochs=20,validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid]))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 76us/sample - loss: 0.7797 - main_output_loss: 0.7130 - aux_output_loss: 1.3810 - val_loss: 1.9584 - val_main_output_loss: 2.0411 - val_aux_output_loss: 1.2064
Epoch 2/20
11610/11610 [==============================] - 0s 39us/sample - loss: 0.5815 - main_output_loss: 0.5513 - aux_output_loss: 0.8527 - val_loss: 0.4904 - val_main_output_loss: 0.4571 - val_aux_output_loss: 0.7910
Epoch 3/20
11610/11610 [==============================] - 0s 40us/sample - loss: 0.4823 - main_output_loss: 0.4526 - aux_output_loss: 0.7495 - val_loss: 0.4590 - val_main_output_loss: 0.4302 - val_aux_output_loss: 0.7184
Epoch 4/20
11610/11610 [==============================] - 0s 39us/sample - loss: 0.4638 - main_output_loss: 0.4383 - aux_output_loss: 0.6939 - val_loss: 0.4476 - val_main_output_loss: 0.4217 - val_aux_output_loss: 0.6816
Epoch 5/20
11610/11610 [==============================] - 0s 40us/s

In [42]:
total_loss,main_loss,aux_loss=model.evaluate([X_test_A,X_test_B],[y_test,y_test])
y_pred_main,y_pred_aux=model.predict([X_new_A,X_new_B])

5160/5160 [==============================] - 0s 22us/sample - loss: 0.3711 - main_output_loss: 0.3556 - aux_output_loss: 0.5019


## Using the Subclassing API to Build Dynamic Models
### Functional APIを試してみよう．

In [49]:
class WideAndDeepModel(keras.Model):
    def __init__(self,units=30,activation="relu",**kwargs):
        super().__init__(**kwargs) #handles standard args (e.g., name)
        self.hidden1=keras.layers.Dense(units,activation=activation)
        self.hidden2=keras.layers.Dense(units,activation=activation)
        self.main_output=keras.layers.Dense(1)
        self.aux_output=keras.layers.Dense(1)
        
    def call(self,inputs):
        input_A,input_B=inputs
        hidden1=self.hidden1(input_B)
        hidden2=self.hidden2(hidden1)
        concat=keras.layers.concatenate([input_A,hidden2])
        main_output=self.main_output(concat)
        aux_output=self.aux_output(hidden2)
        return main_output, aux_output
    
model=WideAndDeepModel()

- 型のチェックなどが事前にできないので，Functional API は不便．極力Sequential APIを活用するべき

### Modelを保存しよう

In [58]:
model.save("my_keras_model.h5")

### ModelをLoadしよう

In [59]:
model=keras.models.load_model("my_keras_model.h5") #roll back to best model 

- save_weights()やload_weights()を使うことでモデルパラメータを保存できるが，それ以外は全て自分で保存する必要がある

### Callbackメソッドを使いチェックポイントで保存する

In [61]:
checkpoint_cb=keras.callbacks.ModelCheckpoint("my_keras_model.h5")
history=model.fit([X_train_A,X_train_B],y_train,epochs=10,callbacks=[checkpoint_cb])

Train on 11610 samples
Epoch 1/10
11610/11610 [==============================] - 0s 32us/sample - loss: 0.4368
Epoch 2/10
11610/11610 [==============================] - 0s 31us/sample - loss: 0.4358
Epoch 3/10
11610/11610 [==============================] - 0s 34us/sample - loss: 0.4349
Epoch 4/10
11610/11610 [==============================] - 0s 30us/sample - loss: 0.4335
Epoch 5/10
11610/11610 [==============================] - 0s 29us/sample - loss: 0.4329
Epoch 6/10
11610/11610 [==============================] - 0s 30us/sample - loss: 0.4316
Epoch 7/10
11610/11610 [==============================] - 0s 31us/sample - loss: 0.4307
Epoch 8/10
11610/11610 [==============================] - 0s 30us/sample - loss: 0.4295
Epoch 9/10
11610/11610 [==============================] - 0s 31us/sample - loss: 0.4287
Epoch 10/10
11610/11610 [==============================] - 0s 32us/sample - loss: 0.4278


In [63]:
early_stopping_cb=keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
history=model.fit(X_train,y_train,epochs=100,validation_data=(X_valid,y_valid),callbacks=[checkpoint_cb,early_stopping_cb])

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), for inputs ['wide_input', 'deep_input'] but instead got the following list of 1 arrays: [array([[-0.84543428, -0.99829321,  0.12468852, ...,  0.01012836,
         2.22308376, -0.51138938],
       [-0.12143853, -0.04555135, -0.30930892, ..., -0.0060048 ,
        -0.81508065,  0.84520221],...